In [ ]:
import sys
from pathlib import Path
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
# set paths to your dataset here
# path_MESMER needs to point to the folder that contains the lfr_scenario_*.nc files 
# path_FAIR needs to point to the folder that contains the scen_*.csv files and that folder also needs to contian the ids_for_sarah.csv 
path_MESMER = Path("/mnt/PROVIDE/mesmer_emulations/mesmer_fair_emulations/fair_100_mesmer_10")
path_FAIR   = Path("/Users/schoens/Documents/Projekte/Econ/Data/FaIR_v3")

In [ ]:
# for a specified scenario, loop over all available lfr_*.nc files,
# select the output from a specific model and calculate the global land mean temperature (glmt)
# as MESMER only introduces a linear scaling in the forced response, the relationship between 
# different glmt should resembe that between global mean temperautre trajectories
scenario = 'CurPol'
# load data 
glmt = np.zeros((100, 251))
for i in tqdm(range(100), total = 100): 
    tas          = xr.load_dataset(path_MESMER / f'lfr_{scenario}_{i}.nc').sel(mesmer_esm_calibration = 'ACCESS-CM2')
    # spatial aggregation
    weights      = np.cos(np.deg2rad(tas.lat))
    weights.name = "weights"
    tas_fldmean  = tas.weighted(weights).mean(("lon", "lat"))
    glmt[i, :]   = tas_fldmean['tas'].values.flatten()

In [ ]:
# from all ~2000 available gmt trajectories, select the 100 that may have been used to 
# generate MESMER data 
fair_ids_100  = pd.read_csv(path_FAIR / 'ids_for_sarah.csv', sep = ',', index_col = 0)
gmt           = pd.read_csv(path_FAIR / f'scen_{scenario}.csv', index_col = 0).loc[slice(1850,2100)].iloc[:, fair_ids_100.loc[:, scenario].values].values.T

In [ ]:
# visually compare estimates; 
# check if the first 5 trajectories are similar
# if not, it might be that the ids are correct, but not in the right oder --> plot the entire ensemble 
# and see if the overall relationships look similar
import matplotlib.pyplot as plt

plt.figure()
plt.plot(np.linspace(1850,2100,251), glmt[:5,:].T)
plt.show()
plt.figure()
plt.plot(np.linspace(1850,2100,251), gmt[:5,:].T)
plt.show()

plt.figure()
plt.plot(np.linspace(1850,2100,251), glmt[:,:].T, color = 'C0',alpha = .2)
plt.show()
plt.figure()
plt.plot(np.linspace(1850,2100,251), gmt[:,:].T, color = 'C0', alpha = .2)
plt.show()